# Section 5: Market Capitalization Model

---

## Model Introduction

### Purpose

This model is meant to forecast the evolution of the market capitalization in discrete time.


### Questions

1. Given ${\mu}$, ${\sigma}$, and ${N}$, what will be the market capitalization after time ${t}$.
2. How do different market conditions affect the market capitalization.


### Assumptions

1. Afropreneur investable capital is normally distributed.
2. Afropreneur attitude toward the JASIRI is positively correlated with their attitude to the general economy and the cryptocurrency economy.
3. There is some relationship between the population and food supply.


### Constraints / Scope

* The intention of this toy model is to allow us to learn about cadCAD, the modelling process, simulation configuration, and the engineering design process!


### Model input

1. The following quantifiable metrics of the afropreneur landscape: number of afropreneurs ${N}$, average afropreneur investable capital ${\mu}$, standard deviation of investable capital ${\sigma}$.


2. Market conditions.


### Model description

Market capitalization shall be modelled using the following:

   1. average amount of afropreneur cash commitment expected, ${\mu}$
   
   
   2. standard deviation of cash commitment expected, ${\sigma}$
   
   
   3. suitable ranges of cash commitments, ${x}$ to ${x+ \Delta x}$ for all ${x}$ in the range of consideration
   
   
   4. the probabilities of getting cash commitments in those ranges, $$ P(x \le X \le x+\Delta x) = \int_{x}^{x+ \Delta x} \frac{1}{\sigma \sqrt(2\pi)} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2} \, dx $$. The Gaussian/ normal probability distribution is used.
   
   
   5. the expected number of afropreneurs to commit per range, ${n}$. ${k \cdot n = N}$ i.e. ${k}$ number of ranges.

The expected amount of cash commitments in each range is then calculated as: $$ \int_{x}^{x+ \Delta x} x \cdot P(X=x) \, dx $$

and the total amount of cash commitments in that range as:$$ n \cdot \int_{x}^{x+ \Delta x} x \cdot P(X=x) \, dx $$

Thus the market capitalization will be calculated as $$ \sum_{j=0}^{k-1} \; n \cdot \int_{x_j}^{x_j+ \Delta x} x \cdot P(X=x) \, dx $$


### Shortcomings of the model

${n}$, the number of afropreneurs per range, is treated as constant in the model. It should eventually be variable subject to the afropreneur landscape that will be defined and refined later, and the varied response to different market conditions along of the spectrum of these afropreneurs.

## 0. Dependencies

In [ ]:
import math

import pandas as pd

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

## 1. State Variables

In [3]:
initial_state = {
    'market_cap': 0
}

# 2. System Parameters

> These are variables that impact the behaviour of the model. They allow consideration of how the system behaves under a different model parameter set (and different parameter values).

In [ ]:
system_params = {
    'number_afropreneurs': 600000
    'avg_investable_capital': [30], # all values in US dollars, unless stated otherwise
    'std_dev_investable_capital': [20],
    'market_conditions': {}
}

# 3. Policy Functions

> A Policy Function computes one or more signals to be passed to State Update Functions. They describe the logic  and behaviour of a system component  or mechanism.

In [ ]:
# utility functions: functions that other constructs in the model make use of, and are abstracted away for simplicity
def normal_dist(x):
    return x * norm.pdf(x, system_params['avg_investable_capital'], system_params['std_dev_investable_capital'])


def p_purchase_simulation(params, substep, state_history, previous_state):
    purchase = 0
    commitment_range = len(range(params['avg_investable_capital']-(2*params['std_dev_investable_capital']),
                             params['avg_investable_capital']+(2*params['std_dev_investable_capital'])))
    afropren_batch_size = 1000  # f(market states)
    cash_batch_size = commitment_range / (600000 / afropren_batch_size)
    lower_cash_value = 0
    for i in range(math.ceil(600000 / afropren_batch_size)):
        purchase += afropren_batch_size * quad(normal_dist, lower_cash_value, lower_cash_value + cash_batch_size)[0]
        lower_cash_value += cash_batch_size
    return {'delta_purchase': purchase}

# 4. State Update Functions

> We create State Update Functions to design the way our model state changes over time.

In [ ]:
def s_update_market_cap(params, substep, state_history, previous_state, policy_input):
    global market_cap
    market_cap = previous_state['market_cap'] + policy_input['delta_purchase']
    return market_cap

In [ ]:
# Relevant state variables
market_cap = initial_state['market_cap']

# Relevant parameters
reproduction_rate = system_params['reproduction_rate'][0] # "alpha" in our differential equation

new_population(current_population, reproduction_rate, food_supply)

In [ ]:
def s_population(params, substep, state_history, previous_state, policy_input):
    """
    Update the population state according to the differential equation (1):
    current_population + alpha * food_supply
    """
    population = previous_state['population']
    alpha = params['reproduction_rate']
    food_supply = previous_state['food']
    
    return 'population', max(new_population(population, alpha, food_supply), 0)

In [ ]:
population = 60
print("A tuple!")
'population', max(math.ceil(population), 0)

In [ ]:
next_state = {
    # current_population + alpha * food_supply
    'population': math.ceil(50 + 0.01 * 1000),
    'food': 1000
}
next_state

In [ ]:
def s_food(params, substep, state_history, previous_state, policy_input):
    """
    Update the food supply state according to the differential equation (2):
    food supply - beta * population
    """
    food = previous_state['food'] - params['consumption_rate'] * previous_state['population']
    return 'food', max(food, 0)

In [ ]:
max(-10, 0)

# 5. Partial State Update Blocks
## Tying it all together

> A series of Partial State Update Blocks is a structure for composing State Update Functions and Policy Functions in series or parallel, as a representation of the system model. 

**Updates run in parallel**

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {
        },
        'variables': {
            'market_cap': s_update_market_cap
        }
    }
]

# 6. Configuration

> The configuration stage is about tying all the previous model components together and choosing how the simulation should run.

<center>
<img src="./images/cadcad-flow.png"
     alt="cadCAD flow"
     style="width: 25%;" />
</center>

Configuration parameters:
* `'N': 1` - the number of times we'll run the simulation (you'll see them called "Monte Carlo runs" later in the course, when we look at tools to analyze system models)
* `'T': range(400)` - the number of timesteps the simulation will run for
* `'M': system_params` - the parameters of the system

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(1),
    "M": system_params
})

In [ ]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [ ]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

# 7. Execution

> The Execution Engine takes a model and configuration, and computes the simulation output.

## Configuring the cadCAD simulation execution

In [ ]:
exec_context = ExecutionContext()

In [ ]:
simulation = Executor(exec_context=exec_context, configs=configs)

## Time to simulate our ecosystem model!

In [ ]:
raw_result, tensor_field, sessions = simulation.execute()

# 8. Simulation Output Preparation
> The simulation results are returned as a list of Python dictionaries, which we then convert to a Pandas dataframe. At this stage of the process you'll manipulate and analyze your results to answer questions about your model.

In [ ]:
simulation_result = pd.DataFrame(raw_result)

In [ ]:
raw_result[:5]

In [ ]:
simulation_result.head()

# 9. Simulation Analysis

[Link to System Requirements](#Requirements-Analysis)

In [ ]:
pd.options.plotting.backend = "plotly"

After plotting the results, let's go and update the parameters, and then select `Cell` and `Run All Above`:

[Link to System Parameters](#2.-System-Parameters)

In [ ]:
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['population','food']
)

In [ ]:
pd.set_option('display.max_rows', len(simulation_result))
display(simulation_result)
pd.reset_option('display.max_rows')

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
simulation_result.query('food == 0').head()

# System Validation

<center><img src="images/edp-phase-3.png" alt="Engineering Design Process, phase 1 - validation" width="60%"/>

## Policy Functions

An illustrative example:

```python
condition = True

def policy_function(params, substep, state_history, previous_state):
    '''
    This logic belongs in the policy function,
    but could also have been placed directly in the state update function.
    '''
    signal_value = 1 if condition else 0
    return {'signal_name': signal_value}
```

```python
def state_update_function(params, substep, state_history, previous_state, policy_input):
    state_value = policy_input['signal_name']
    return 'state_name', state_value
```

<center>
<img src="./images/basic-psub.png"
     alt="Policy functions"
     style="width: 60%;" />
</center>

<center>
<img src="./images/policy-explainer.png"
     alt="Policy functions"
     style="width: 60%;" />
</center>

### Policy Aggregation

<center>
<img src="./images/policy-aggregation-explainer.png"
     alt="Policy functions"
     style="width: 60%;" />
</center>

## Model Improvements

### Differential Specification Updates

<center>
<img src="./images/s5-differential-spec-ecosystem-policy-refactor.png"
     alt="Differential spec"
     style="width: 60%;" />
</center>

State update functions `s_population()` and `s_food()` from the last part:

In [ ]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + params['reproduction_rate'] * previous_state['food']
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] - params['consumption_rate'] * previous_state['population']
    return 'food', max(food, 0)

Adapting to use **policy functions** to drive the process, and **state update functions** to update the state according to the **differential specification**:

In [ ]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

In [ ]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food'] 
    return 'food', max(food, 0)

### Mathematical Specification Updates

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(sheep)} \tag{1} \\
\large food_t &\large= food_{t-1} + {\Delta food} \quad \textrm{(tons of grass)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * food_{t-1} \quad \textrm{(sheep/month)} \\
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma \quad \textrm{(tons of grass/month)}
\end{align}

where:

$
\begin{align}
\alpha: \quad &\textrm{'reproduction_rate'}\\
\beta: \quad &\textrm{'consumption_rate'}\\
\gamma: \quad &\textrm{'growth_rate'}
\end{align}
$

* A population consumes a food source, and reproduces at a rate proportional to the food source $\alpha$ (alpha).
* The food source is consumed at a rate proportional to the population $\beta$ (beta), and grows at a constant rate $\gamma$ (gamma).

<center>
<img src="./images/s6-differential-spec-ecosystem-with-gamma.png"
     alt="Diff spec"
     style="width: 60%" />
</center>

In [ ]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'reproduction_rate': [0.01], # number of sheep / month
    'consumption_rate': [0.01], # tons of grass / month
    'growth_rate': [10.0], # tons of grass / month
}

In [ ]:
from collections import Counter

In [ ]:
A = Counter({'delta_food': 5, 'delta_population': 10})
B = Counter({'delta_food': 5})
A + B

In [ ]:
A = Counter({'delta_food': 5, 'delta_population': 10})
B = Counter({'delta_food': -2})
A + B

In [ ]:
def p_growth(params, substep, state_history, previous_state):
    delta_food = params['growth_rate']
    return {'delta_food': delta_food}

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'consumption': p_consumption, # Signal: `delta_food`
            'growth': p_growth # Signal: `delta_food`
        },
        'variables': {
            'population': s_population,
            'food': s_food # Receives policy_input of (consumption + growth) as `delta_food`
        }
    }
]

In [ ]:
del configs[:]

sim_config = config_sim({
    'N': 1,
    'T': range(400),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

In [ ]:
df = simulation_result.copy()
df = df[df.simulation == 0]
df

In [ ]:
df.plot(kind='line', x='timestep', y=['population','food'])

In [ ]:
df = df[['population', 'food']]
df.head()

In [ ]:
df.pct_change()

In [ ]:
diff = df.diff()
diff

In [ ]:
diff = diff.query('food <= 0')
diff

In [ ]:
df.iloc[75]

## Model Limitations

1. The population never dies.
2. The system reaches a steady state of no population or food supply change.

#### Addition of a population death rate, "epsilon" / $\epsilon$, that's dependent on the population size:
<br>

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(sheep)} \tag{1} \\
\large food_t &\large= food_{t-1} + {\Delta food} \quad \textrm{(tons of grass)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * food_{t-1} - \epsilon * population_{t-1} \quad \textrm{(sheep/month)} \\
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma \quad \textrm{(tons of grass/month)}
\end{align}

where:

$
\begin{align}
\alpha: \quad &\textrm{'reproduction_rate'}\\
\epsilon: \quad &\textrm{'death_rate'}\\
\beta: \quad &\textrm{'consumption_rate'}\\
\gamma: \quad &\textrm{'growth_rate'}\\
\end{align}
$

* A population consumes a food source, and reproduces at a rate proportional to the food source $\alpha$ (alpha), and dies at a rate proportional to the population size $\epsilon$ (epsilon).
* The food source is consumed at a rate proportional to the population $\beta$ (beta), and grows at a constant rate $\gamma$ (gamma).

<center>
<img src="./images/s6-differential-spec-ecosystem-final.png"
     alt="Diff spec"
     style="width: 60%" />
</center>

In [ ]:
def p_death(params, substep, state_history, previous_state):
    population_death = params['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

In [ ]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'reproduction_rate': [0.01],
    'death_rate': [0.01],
    'consumption_rate': [0.01],
    'growth_rate': [10.0],
}

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [ ]:
sim_config = config_sim({
    'N': 1,
    'T': range(1000),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)

In [ ]:
df = simulation_result.copy()
df = df[df.simulation == 1]
df

In [ ]:
df.plot(kind='line', x='timestep', y=['population','food'])

<br/><br/><br/>
# Well done!
<br/><br/><br/><br/>